# United States Weather Bureau Method for Evaporation

This method is based on the Penman equation for evaporation, and is dependant on the daily average air temperature, relative humidity, daily solar radiation, and the average daily wind speed

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
hollis = pd.read_csv("Hollis.csv")
hollis.head()

,Date / Time,AirTC (Avg),RH,SlrkW (Avg),SlrjJ (Tot),WS ms (Avg),Wind Dir,T108 C (Avg),HBr3W,Rain mm (Tot)
0,7/17/2014 15:30,26.94,67.85,0.412,609.3776,1.667,335.900,31.21,221.5,0.0
1,7/17/2014 16:00,26.89,69.88,0.387,696.9075,1.618,59.220,31.45,221.4,0.0
2,7/17/2014 16:30,26.45,73.05,0.154,277.9549,1.753,52.930,31.25,221.3,0.0
3,7/17/2014 17:00,26.27,73.78,0.183,328.9092,1.879,2.172,30.91,220.9,0.0
4,7/17/2014 17:30,25.95,75.34,0.060,107.9211,1.944,33.750,30.47,220.8,0.0


In [3]:
df = hollis[["Date / Time", "RH", "SlrjJ (Tot)", "WS ms (Avg)", "T108 C (Avg)"]]    #Using pan temp since air temp has some weird values
df.columns = ['date', 'humidity', "energy", "wind", 'temp']
df.dropna(inplace=True)
df["date"] = pd.to_datetime(df.date).dt.date
df.describe()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,humidity,energy,wind,temp
count,20054.000000,20054.000000,20054.000000,20054.000000
mean,80.269694,314.413236,1.167754,27.002141
std,54.656236,473.302872,0.797486,2.911931
min,0.612000,0.000000,0.000000,20.460000
25%,7.816250,0.000000,0.429000,24.770000
50%,91.100000,5.817208,1.087000,26.440000
75%,119.000000,528.021650,1.739000,28.960000
max,217.200000,2203.842000,4.445000,37.270000


In [4]:
df.head()

,date,humidity,energy,wind,temp
0,2014-07-17,67.85,609.3776,1.667,31.21
1,2014-07-17,69.88,696.9075,1.618,31.45
2,2014-07-17,73.05,277.9549,1.753,31.25
3,2014-07-17,73.78,328.9092,1.879,30.91
4,2014-07-17,75.34,107.9211,1.944,30.47


In [5]:
df = df.groupby('date', as_index=False, sort=False).mean()
df.head()

,date,humidity,energy,wind,temp
0,2014-07-17,81.728235,122.398876,1.359412,28.745294
1,2014-07-18,89.427083,105.583912,1.032292,24.944583
2,2014-07-19,86.988125,227.233568,1.284333,25.877708
3,2014-07-20,88.730833,172.929764,1.010750,25.772083
4,2014-07-21,82.476458,367.473248,1.176583,27.385000


## Relative Humidity

First, a component of relative humidity is calculated based on the the average daily relative humidity f, in percent.

\begin{equation*}
X = 1 - {f\over 100}
\end{equation*}

In [6]:
df['CRH'] = 1 - (df['humidity'])/100
df.head()

,date,humidity,energy,wind,temp,CRH
0,2014-07-17,81.728235,122.398876,1.359412,28.745294,0.182718
1,2014-07-18,89.427083,105.583912,1.032292,24.944583,0.105729
2,2014-07-19,86.988125,227.233568,1.284333,25.877708,0.130119
3,2014-07-20,88.730833,172.929764,1.010750,25.772083,0.112692
4,2014-07-21,82.476458,367.473248,1.176583,27.385000,0.175235


## Dewpoint Temperature

\begin{equation*}
T_d = T_a - \left[(14.55 + 0.114T_a)X + [(2.5 + 0.007T_a)X]^3 + (15.9 + 0.117T_a)X^{14}\right]
\end{equation*}

where T<sub>a</sub> is the average daily temperature in degrees Celsius

In [7]:
df['DT'] = df['temp']-(14.55+0.114*df['temp'])*df['CRH']+((2.5+0.007*df['temp'])*df['CRH'])**3+(15.9+0.117*df['temp'])*df['CRH']**14
df.head()

,date,humidity,energy,wind,temp,CRH,DT
0,2014-07-17,81.728235,122.398876,1.359412,28.745294,0.182718,25.608225
1,2014-07-18,89.427083,105.583912,1.032292,24.944583,0.105729,23.128177
2,2014-07-19,86.988125,227.233568,1.284333,25.877708,0.130119,23.643083
3,2014-07-20,88.730833,172.929764,1.010750,25.772083,0.112692,23.828889
4,2014-07-21,82.476458,367.473248,1.176583,27.385000,0.175235,24.393200


## Dimensionless Ratio

\begin{equation*}
{\Delta \over{\Delta + \gamma}} = \left[1+{0.66\over{(0.00815T_a + 0.8912)^7}}\right]^{-1}
\end{equation*}

Just do it

\begin{equation*}
{\gamma \over{\Delta + \gamma}} = {1 - {\Delta\over{\Delta+\gamma}}}
\end{equation*}

In [8]:
df['ratio'] = 1-(1+(0.66/(0.00815*df['temp']+0.8912)**7))**-1
df.head()

,date,humidity,energy,wind,temp,CRH,DT,ratio
0,2014-07-17,81.728235,122.398876,1.359412,28.745294,0.182718,25.608225,0.223924
1,2014-07-18,89.427083,105.583912,1.032292,24.944583,0.105729,23.128177,0.259688
2,2014-07-19,86.988125,227.233568,1.284333,25.877708,0.130119,23.643083,0.250479
3,2014-07-20,88.730833,172.929764,1.010750,25.772083,0.112692,23.828889,0.251507
4,2014-07-21,82.476458,367.473248,1.176583,27.385000,0.175235,24.393200,0.236196


## Effective Net Radiation

\begin{equation*}
Q_n = 0.00714Q_s + 0.00000526Q_s(T_a + 17.8)^{1.87} + 0.00000394Q_s^2 - 0.00000000239Q_s^2(T_a-7.2)^2-1.02
\end{equation*}

where Q<sub>s</sub> is the daily solar radiation in calories per square centimeter. We need to do a conversion

In [9]:
df['rad'] = 0.00714*df['energy']+0.00000526*df['energy']*(df['temp']+17.8)**1.87+0.000000394*df['energy']**2-0.00000000239*df['energy']**2*(df['temp']-7.2)**2-1.02
df.head()

,date,humidity,energy,wind,temp,CRH,DT,ratio,rad
0,2014-07-17,81.728235,122.398876,1.359412,28.745294,0.182718,25.608225,0.223924,0.689834
1,2014-07-18,89.427083,105.583912,1.032292,24.944583,0.105729,23.128177,0.259688,0.352647
2,2014-07-19,86.988125,227.233568,1.284333,25.877708,0.130119,23.643083,0.250479,1.975286
3,2014-07-20,88.730833,172.929764,1.010750,25.772083,0.112692,23.828889,0.251507,1.259092
4,2014-07-21,82.476458,367.473248,1.176583,27.385000,0.175235,24.393200,0.236196,3.930113


## Vapour Pressure Difference

\begin{equation*}
e_s-e_a=33.86\left[(0.00738T_a+0.8072)^8 - (0.00738T_d + 0.8072)^8\right]
\end{equation*}

where v<sub>p</sub> is the average wind speed in km per day

In [10]:
df['vapour'] = 33.86*((0.00738*df['temp']+0.8072)**8-(0.00738*df['DT']+0.8072)**8)
df.head()

,date,humidity,energy,wind,temp,CRH,DT,ratio,rad,vapour
0,2014-07-17,81.728235,122.398876,1.359412,28.745294,0.182718,25.608225,0.223924,0.689834,6.626309
1,2014-07-18,89.427083,105.583912,1.032292,24.944583,0.105729,23.128177,0.259688,0.352647,3.258145
2,2014-07-19,86.988125,227.233568,1.284333,25.877708,0.130119,23.643083,0.250479,1.975286,4.163760
3,2014-07-20,88.730833,172.929764,1.010750,25.772083,0.112692,23.828889,0.251507,1.259092,3.627889
4,2014-07-21,82.476458,367.473248,1.176583,27.385000,0.175235,24.393200,0.236196,3.930113,5.913547


## Evaporation

\begin{equation*}
E_a=(e_s-e_a)^{0.88}(0.42+0.0029v_p)
\end{equation*}

In [11]:
df['evap'] = df['vapour']**0.88*(0.42+0.0029*df['wind'])
df.head()

,date,humidity,energy,wind,temp,CRH,DT,ratio,rad,vapour,evap
0,2014-07-17,81.728235,122.398876,1.359412,28.745294,0.182718,25.608225,0.223924,0.689834,6.626309,2.238854
1,2014-07-18,89.427083,105.583912,1.032292,24.944583,0.105729,23.128177,0.259688,0.352647,3.258145,1.196046
2,2014-07-19,86.988125,227.233568,1.284333,25.877708,0.130119,23.643083,0.250479,1.975286,4.163760,1.486726
3,2014-07-20,88.730833,172.929764,1.010750,25.772083,0.112692,23.828889,0.251507,1.259092,3.627889,1.314514
4,2014-07-21,82.476458,367.473248,1.176583,27.385000,0.175235,24.393200,0.236196,3.930113,5.913547,2.022970


## Monthly Evaporation

In [12]:
month_df = df[['date', 'evap']]
month_df.set_index('date', inplace=True)
month_df.head()

,evap
date,
2014-07-17,2.238854
2014-07-18,1.196046
2014-07-19,1.486726
2014-07-20,1.314514
2014-07-21,2.022970


In [13]:
month_df.describe()

,evap
count,162.000000
mean,1.587357
std,0.589509
min,0.011408
25%,1.279294
50%,1.667830
75%,1.975631
max,3.554935


In [15]:
month_df.dtypes

evap    float64
dtype: object

In [14]:
month_df_sum = month_df.groupby(pd.TimeGrouper("M")).sum()
month_df_sum.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  """Entry point for launching an IPython kernel.


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [ ]:
month_df_sum.plot()

## References

https://pubs.usgs.gov/sir/2012/5202/pdf/sir2012-5202.pdf